In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/asmolka/repos/exess/smolkaa.github.io/manuals/code_examples/example_trajectory_01"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| echo: false
#| output: false
path_to_exess = joinpath(@__DIR__, "..", "..", "..", "..", "exess.jl", "src", "ExESS.jl")

"/home/asmolka/repos/exess/smolkaa.github.io/manuals/code_examples/example_trajectory_01/../../../../exess.jl/src/ExESS.jl"

In [3]:
versioninfo()

Julia Version 1.9.3
Commit bed2cd540a1 (2023-08-24 14:43 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 8 × Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, skylake)
  Threads: 2 on 8 virtual cores


In [4]:
#| output: false
include(path_to_exess)
using .ExESS

In [5]:
x0 = GlobalCartesianPosition(LUNAR_RADIUS, 0, 0)

GlobalCartesianPosition{Float64}(1.7374e6, 0.0, 0.0)

In [6]:
v0 = GlobalCartesianVelocity(100, 0, 0)

GlobalCartesianVelocity{Int64}(100, 0, 0)

In [7]:
ddx = ddx_lunar_gravity

ddx_lunar_gravity (generic function with 2 methods)

In [8]:
traj = trajectory(x0, v0, ddx)

retcode: Terminated
Interpolation: specialized 4th order "free" interpolation
t: 7-element Vector{Float32}:
   0.0
   0.17253026
   1.897833
  19.15086
  89.02362
 125.11684
 125.11684
u: 7-element Vector{RecursiveArrayTools.ArrayPartition{Float64, Tuple{Vector{Float64}, Vector{Float64}}}}:
 ([100.0, 0.0, 0.0], [1.737401e6, 0.0, 0.0])
 ([99.72352819548777, 0.0, 0.0], [1.7374182291764107e6, 0.0, 0.0])
 ([96.95910868428572, 0.0, 0.0], [1.737587897640642e6, 0.0, 0.0])
 ([69.34166361242546, 0.0, 0.0], [1.7390224279588673e6, 0.0, 0.0])
 ([-42.27453422294352, 0.0, 0.0], [1.739967349345177e6, 0.0, 0.0])
 ([-100.0160468921404, 0.0, 0.0], [1.7374e6, 0.0, 0.0])
 ([-100.0160468921404, 0.0, 0.0], [1.7374e6, 0.0, 0.0])